# Health Database Query

This project creates a database that stores world health and population information collected between July 8th and November 8th 2012, and queries the database for relevant disease information.

## 1. Create health database
Create `healthData.db` and store the disease information from `table.csv` and `healthfeed.json`.

In [1]:
import sqlite3
import json
import csv
import sys

In [2]:
con = sqlite3.connect("healthData.db")
con.text_factory = str
cur = con.cursor()
commit_point = 5000

In [3]:
def create_database():
    cur.execute('drop table if exists health_csv')
    cur.execute("""create table health_data_csv
                    (Country text, Location text, Latitude real, Longitude real, Disease text, Report_date text, 
                     Summary text, Description text, Rating integer, News_feed text, Link text) 
                 """)
    cur.execute('drop table if exists health_json')
    cur.execute("""create table health_data_json
                    (Country text, Location text, Latitude real, Longitude real, Disease text, Report_date text, 
                     Summary text, Description text, Rating integer, News_feed text, Link text) 
                 """)

In [4]:
def insert(data, ext):
    if data and len(data) > 0:
        cur.executemany("insert into health_data" + ext + " VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data)
        con.commit()

In [5]:
def load_data(filename, load_format):
    if load_format == 'json':
        load_json(filename)
    elif load_format == 'csv':
        load_csv(filename)
    else:
        print 'ERROR: Invalid format, please specify 1 of "json" or "csv".'

In [6]:
def load_json(filename):
    json_file = open(filename, 'rU')
    i = 0
    data = []
    for line in json_file:
        entries = json.loads(line)
        for entry in entries:
            for alert in entry['alerts']:
                data.append((entry['country'], entry['place_name'], float(entry['lat']), float(entry['lng']), alert['disease'], alert['date'], alert['summary'], alert['descr'], int(float(alert['rating']['rating'])), alert['feed'], alert['link']))
                i += 1
                if i % commit_point == 0:
                    insert(data, '_json')
                    data = []
    json_file.close()
    insert(data, '_json')

In [7]:
def load_csv(filename):
    with open(filename, 'rU') as f:
        reader = csv.reader(f, delimiter=',')
        i = 0
        data = []
        is_first = True
        for row in reader:
            if is_first:
                is_first = False
                continue
            data.append((row[0], row[1], float(row[2]), float(row[3]), row[4], row[5], row[6], row[7], int(float(row[8])), row[9], row[10]))
            i += 1
            if i % commit_point == 0:
                insert(data, '_csv')
                data = []
        insert(data, '_csv')

In [8]:
def main():
    try:
        create_database()
        load_data('healthfeed.json', 'json')
        load_data('table.csv', 'csv')
    except sqlite3.Error, e:
        print "Error %s:" % e.args[0]
        sys.exit(1)
    finally:
        if con:
            con.commit()
            con.close()

In [9]:
if __name__ == '__main__':
    main()

## 2. Add an outbreak table to the health database
Create a table `outbreak` in `healthData.db` that store disease information for each country by the date and number of occurrences.

In [10]:
import sqlite3 as sqlite
import sys

In [11]:
#Get the data from the main database
with sqlite.connect('healthData.db') as con:  
    cur = con.cursor() 
    cur.execute('''select country, disease, date(report_date) as short_date, count(*), avg(rating)
                from health_data_csv
                group by country, disease, date(report_date)''')
    rows = cur.fetchall()
    
    #Dump results into "outbreaks" table that we can subsequently use for outbreak-based analysis  
    cur.execute("drop table if exists outbreaks")
    cur.execute("create table outbreaks(Country text, Disease text, Date text, Report_instances integer, Average_rating float)")
    cur.executemany("insert into outbreaks values (?, ?, ?, ?, ?)", rows)

## 3. Add a population table to the health database
Create a table `populations` in `healthData.db` that store 2012 United Nation population estimates from `population12.csv`. Note: The population figures are in 1000s.

In [12]:
import sys, csv
import sqlite3 as sqlite

In [13]:
def unicode_escape(x):
    try:
        return unicode(x)
    except UnicodeDecodeError:
        byte_text = str(x).encode('string_escape')
        return unicode(byte_text)

In [14]:
with open('population12.csv','rU') as csvfile:
    popreader = csv.reader(csvfile)
    data = []
    for row in popreader:
        x = row[65].split(',')
        pop = ''.join(x)
        l =(unicode_escape(row[0]), unicode_escape(row[1]), float(pop))
        data.append(l) 

In [15]:
with sqlite.connect('healthData.db') as con: 
    cur = con.cursor() 
    cur.execute("drop table if exists populations")
    cur.execute("create table populations(Country text, ISO_Code text, Population_2012 FLOAT)")
    cur.executemany("insert into populations values (?, ?, ?)", data)

## 4. Query the health database for relevant disease information

Query `healthData.db` for the following information and output the queries into txt files:

1) Disease outbreak counts in each country
2) Disease outbreak counts in each country by disease type
3) Average rating and number of outbreaks by country
4) Disease frequencies by Day of Week
5) Total disease counts by Day of Week
6) Outbreaks per capita for each country
7) Total outbreaks for each disease
8) Average and max number reports per disease outbreak per country
9) Reports per capita for a given country
10) Top 10 countries by disease outbreaks

In [16]:
import sqlite3 as sqlite
import sys

In [17]:
try: 
    con = sqlite.connect('healthData.db')
    cur = con.cursor()
    
    # Query 1: Country outbreak counts
    cur.execute('''select Country, count(Disease) as Outbreak_Counts 
                from outbreaks 
                group by Country''')
    query1 = open("country outbreak counts.txt",'w')
    query1.write("Disease outbreak counts in each country:" + "\n")
    query1.write("Country" + "\t" + "Outbreak counts" + "\n")
    query1.write("-------------------------")
    print "%s" % ("Disease outbreak counts in each country:")
    print "%s\t%s" % ("Country", "Outbreak counts")
    print "-------------------------"
    for row in cur:
        query1.write("\n%s\t%s" %(row[0],row[1]))
        print "%s\t%s" % (row[0],row[1])
    
    print
    
    # Query 2: Disease outbreak counts in each country by disease type
    cur.execute('''select Country, Disease, count(*) as Outbreak_Counts 
                from outbreaks 
                group by Country, Disease 
                order by Country, count(*) desc''')
    query2 = open("country diseasetype frequency.txt",'w')
    query2.write("Disease outbreak counts in each country by disease type:" + "\n")
    query2.write("Country" + "\t" + "Disease" + "\t" + "Outbreak counts" + "\n")
    query2.write("--------------------------------")
    print "%s" % ("Disease outbreak counts in each country by disease type:")
    print "%s\t%s\t%s" % ("Country", "Disease", "Outbreak counts")
    print "--------------------------------"
    for row in cur:
        query2.write("\n%s\t%s\t%s" %(row[0],row[1],row[2]))
        print "%s\t%s\t%s" % (row[0],row[1],row[2])
    
    print
    
    # Query 3: Average rating and number of outbreaks by country
    cur.execute('''select Country, avg(Average_rating) as Average_ratings, count(*) as Num_Outbreaks 
                from outbreaks 
                group by Country''')
    query3 = open("country rating.txt",'w')
    query3.write("Average rating and number of outbreaks by country:" + "\n")
    query3.write("Country" + "\t" + "Average rating" + "\t" + "Outbreak counts" + "\n")
    query3.write("----------------------------------------")
    print "%s" % ("Average rating and number of outbreaks by country:")
    print "%s\t%s\t%s" % ("Country", "Average rating", "Outbreak counts")
    print "----------------------------------------"
    for row in cur:
        query3.write("\n%s\t%s\t%s" %(row[0],row[1],row[2]))
        print "%s\t%s\t%s" % (row[0],row[1],row[2])
    
    print
    
    # Query 4: Disease distribution by Day of Week
    cur.execute('''select case 
                        when strftime('%w', Report_date) = '0' then 'Sunday' 
                        when strftime('%w', Report_date) = '1' then 'Monday' 
                        when strftime('%w', Report_date) = '2' then 'Tuesday' 
                        when strftime('%w', Report_date) = '3' then 'Wednesday' 
                        when strftime('%w', Report_date) = '4' then 'Thursday' 
                        when strftime('%w', Report_date) = '5' then 'Friday' 
                        when strftime('%w', Report_date) = '6' then 'Saturday' 
                        else strftime('%w', Report_date) end as DOW, Disease, count(*) as Disease_Counts 
                from health_data_csv 
                where date(Report_date) between date('2012-08-05') and ('2012-10-27') 
                group by strftime('%w', Report_date), Disease''')
    query4 = open("dow disease distribution.txt",'w')
    query4.write("Disease distribution by Day of Week:" + "\n")
    query4.write("DOW" + "\t" + "Disease" + "\t" + "Disease counts" + "\n")
    query4.write("---------------------------------")
    print "%s" % ("Disease distribution by Day of Week:")
    print "%s\t%s\t%s" % ("DOW", "Disease", "Disease counts")
    print "---------------------------------"
    for row in cur:
        query4.write("\n%s\t%s\t%s" %(row[0],row[1],row[2]))
        print "%s\t%s\t%s" % (row[0],row[1],row[2])
    
    print
    
    # Query 5: Disease counts by Day of Week
    cur.execute('''select case 
                        when strftime('%w', Report_date) = '0' then 'Sunday' 
                        when strftime('%w', Report_date) = '1' then 'Monday' 
                        when strftime('%w', Report_date) = '2' then 'Tuesday' 
                        when strftime('%w', Report_date) = '3' then 'Wednesday' 
                        when strftime('%w', Report_date) = '4' then 'Thursday' 
                        when strftime('%w', Report_date) = '5' then 'Friday' 
                        when strftime('%w', Report_date) = '6' then 'Saturday' 
                        else strftime('%w', Report_date) 
                        end as DOW, count(*) as Disease_Counts 
                from health_data_csv 
                where date(Report_date) between date('2012-08-05') and ('2012-10-27') 
                group by strftime('%w', Report_date)''')
    query5 = open("dow counts.txt",'w')
    query5.write("Disease counts by Day of Week:" + "\n")
    query5.write("DOW" + "\t" + "Disease counts" + "\n")
    query5.write("-----------------------")
    print "%s" % ("Disease counts by Day of Week:")
    print "%s\t%s" % ("DOW", "Disease counts")
    print "-----------------------"
    for row in cur:
        query5.write("\n%s\t%s" %(row[0],row[1]))
        print "%s\t%s" % (row[0],row[1])
    
    print
    
    # Query 6: Outbreaks per capita for each country
    cur.execute('''select o.Country, count(*) as Incidents, p.Population_2012, (count(*)/p.Population_2012) as Incidents_per_capita 
                from outbreaks as o 
                join populations as p on (o.Country = p.Country) 
                group by o.Country 
                order by Incidents_per_capita desc''')
    query6 = open("outbreaks per capita.txt",'w')
    query6.write("Outbreaks per capita for each country:" + "\n")
    query6.write("Country" + "\t" + "Incidents" + "\t" + "Population 2012 (in 1000s)" + "\t" + "Incidents per capita" + "\n")
    query6.write("-----------------------------------------------------------------------------")
    print "%s" % ("Outbreaks per capita for each country:")
    print "%s\t%s\t%s\t%s" % ("Country", "Incidents", "Population 2012 (in 1000s)", "Incidents per capita")
    print "-----------------------------------------------------------------------------"
    for row in cur:
        query6.write("\n%s\t%s\t%s\t%s" %(row[0],row[1],row[2],row[3]))
        print "%s\t%s\t%s\t%s" % (row[0],row[1],row[2],row[3])
    
    print 
    
    # Query 7: Rank disease incidents
    cur.execute('''select Disease, count(*) as Outbreak_counts, count(distinct Country) as Num_Countries, sum(Report_instances) as Total_Reports, 
                  cast(sum(Report_instances)/count(*) as float) as Reports_per_Outbreak 
                from outbreaks group by Disease 
                order by count(*) desc''')
    query7 = open("rank diseases.txt",'w')
    query7.write("Rank disease incidents:" + "\n")
    query7.write("Disease" + "\t" + "Outbreak counts" + "\t" + "Number of countries" + "\t" + "Total reports" + "\t" + "Reports per outbreak" + "\n")
    query7.write("--------------------------------------------------------------------------------------")
    print "%s" % ("Rank disease incidents:")
    print "%s\t%s\t%s\t%s\t%s" % ("Disease", "Outbreak counts", "Number of countries", "Total reports", "Reports per outbreak")
    print "--------------------------------------------------------------------------------------"
    for row in cur:
        query7.write("\n%s\t%s\t%s\t%s\t%s" %(row[0],row[1],row[2],row[3],row[4]))
        print "%s\t%s\t%s\t%s\t%s" % (row[0],row[1],row[2],row[3],row[4])
    
    print
    
    # Query 8: Average and max number reports per disease outbreak per country
    cur.execute('''select Country, sum(Report_instances) as Total_Reports, count(*) as Total_Outbreaks, 
                  avg(Report_instances) as Avg_Reports_per_Outbreak, max(Report_instances) as Max_Report_Instances 
                from outbreaks 
                group by Country''')
    query8 = open("report numbers.txt",'w')
    query8.write("Average and max number reports per disease outbreak per country:" + "\n")
    query8.write("Country" + "\t" + "Total reports" + "\t" + "Total outbreaks" + "\t" + "Average reports per outbreak" + "\t" + "Max report instances" + "\n")
    query8.write("---------------------------------------------------------------------------------------------")
    print "%s" % ("Average and max number reports per disease outbreak per country:")
    print "%s\t%s\t%s\t%s\t%s" % ("Country", "Total reports", "Total outbreaks", "Average reports per outbreak", "Max report instances")
    print "---------------------------------------------------------------------------------------------"
    for row in cur:
        query8.write("\n%s\t%s\t%s\t%s\t%s" %(row[0],row[1],row[2],row[3],row[4]))
        print "%s\t%s\t%s\t%s\t%s" % (row[0],row[1],row[2],row[3],row[4])
    
    print
    
    # Query 9: Reports per capita for a given country
    cur.execute('''select h.Country, count(*) as Reports, p.Population_2012,(count(*)/p.Population_2012) as Reports_country_per_capita 
                from health_data_csv as h 
                join populations as p on (h.Country = p.Country) 
                group by h.Country 
                order by Reports_country_per_capita desc''')
    query9 = open("reports per capita.txt",'w')
    query9.write("Reports per capita for a given country:" + "\n")
    query9.write("Country" + "\t" + "Total reports" + "\t" + "Population 2012 (in 1000s)" + "\t" + "Reports per capita" + "\n")
    query9.write("---------------------------------------------------------------------------")
    print "%s" % ("Reports per capita for a given country:")
    print "%s\t%s\t%s\t%s" % ("Country", "Total reports", "Population 2012 (in 1000s)", "Reports per capita")
    print "---------------------------------------------------------------------------"
    for row in cur:
        query9.write("\n%s\t%s\t%s\t%s" %(row[0],row[1],row[2],row[3]))
        print "%s\t%s\t%s\t%s" % (row[0],row[1],row[2],row[3])
    
    print
    
    # Query 10: Top 10 outbreak countries
    cur.execute('''select Country, count(*) as Outbreak_counts 
                from outbreaks 
                group by Country 
                order by count(*) desc limit 10''')
    query10 = open("top 10 outbreak countries.txt",'w')
    query10.write("Top 10 outbreak countries:" + "\n")
    query10.write("Country" + "\t" + "Total outbreak counts" + "\n")
    query10.write("------------------------------")
    print "%s" % ("Top 10 outbreak countries:")
    print "%s\t%s" % ("Country", "Total outbreak counts")
    print "------------------------------"
    for row in cur:
        query10.write("\n%s\t%s" %(row[0],row[1]))
        print "%s\t%s" % (row[0],row[1])
    
    print 
    
    # Query 11: Print all Health web links
    cur.execute("select Link from health_data_csv")
    query11 = open("links_source.txt",'w')
    for row in cur:
        query11.write("\n%s" %(row[0]))

except sqlite.Error, e:
    print "Error %s:" % e.args[0]
    sys.exit(1)

finally:
    if con:
        con.close()

Disease outbreak counts in each country:
Country	Outbreak counts
-------------------------
Afghanistan	5
Albania	5
Algeria	16
Angola	28
Argentina	131
Armenia	7
Australia	81
Austria	5
Azerbaijan	4
Bahrain	2
Bangladesh	3
Barbados	1
Belarus	2
Belgium	15
Benin	4
Bhutan	11
Bolivia	25
Brazil	216
Bulgaria	2
Burkina Faso	1
Burundi	1
Cambodia	19
Cameroon	3
Canada	243
Cape Verde	2
Central African Republic	1
Chad	5
Chile	44
China	335
Colombia	45
Costa Rica	21
Cote d'Ivoire	5
Croatia	5
Cuba	44
Czech Republic	7
DR Congo	82
Denmark	7
Dominican Republic	72
Ecuador	12
Egypt	104
El Salvador	52
Estonia	5
Ethiopia	4
Fiji	3
Finland	3
France	57
Gambia	5
Germany	52
Ghana	23
Greece	28
Guatemala	15
Guinea	30
Guinea-Bissau	3
Guyana	1
Haiti	20
Honduras	68
Hungary	6
Iceland	3
India	244
Indonesia	35
Iran	4
Iraq	13
Ireland	16
Israel	23
Italy	24
Jamaica	11
Japan	20
Jordan	13
Kazakhstan	10
Kenya	38
Kuwait	3
Kyrgyzstan	6
Laos	1
Latvia	4
Lebanon	2
Liberia	1
Libya	1
Madagascar	8
Malawi	6
Malaysia	17
Mali	7
Malta	2
Mars

## 5. Analysis of health web links
Analyze the distribution of languages, domains and TLDs from the health web links `links.txt`.

In [18]:
from operator import itemgetter
import collections, urllib, tldextract, re, logging

In [19]:
logging.basicConfig()

language = []
tlds = []
domains = []
langtlds = []
langdomains = []

Takes a list and turns it into a dictionary of counts

In [20]:
def make_counter(data):
    count = collections.Counter()
    for item in data:
        count[item] += 1
    count_list = []
    for key in count:
        count_list.append([key,count[key]])
    count_list=sorted(count_list, key=itemgetter(1),reverse=True)
    return count_list

Take a dictionary and outputs it to a txt file

In [21]:
def write_counter(data,name):
    path = name + '.txt'
    with open(path,'wb') as f:
        for item in data:
            if isinstance(item[0],str):
                row = item[0] + '\t' + str(item[1]) + '\n'
            else:
                row = item[0][0] + '\t' + item[0][1] + '\t'+ str(item[1]) + '\n'
            f.write(row)

In [22]:
'''
# Open every url from link_source.txt and returns a file of the redirect links
# *Long* process, do not uncomment.

i=
with open('links_source.txt','r') as f:
    for row in f:
        try:
            print row
            link = urllib.urlopen(row).geturl()
            urllib.urlopen(row).close()
            links.append(link)
        except:
            links.append('')
            print i

with open('links.txt','a') as f:
    for item in links:
        f.write(item)
        f.write('\n')
        links = []
'''

"\n# Open every url from link_source.txt and returns a file of the redirect links\n# *Long* process, do not uncomment.\n\ni=\nwith open('links_source.txt','r') as f:\n    for row in f:\n        try:\n            print row\n            link = urllib.urlopen(row).geturl()\n            urllib.urlopen(row).close()\n            links.append(link)\n        except:\n            links.append('')\n            print i\n\nwith open('links.txt','a') as f:\n    for item in links:\n        f.write(item)\n        f.write('\n')\n        links = []\n"

Process the links for lanuages, TLDs, and domains into lists

In [25]:
with open('links.txt','r') as f:
    for row in f:
        row = row.rstrip('\n')
        if re.search('translate',row):
            lang_code=row[-2:]
            if re.search(r'news\.google',row):
                url_match = re.search(r'%3D(http.+)',row)
            else:
                url_match = re.search(r'=(http.+)',row)
            url = urllib.unquote(url_match.group(1))
            tldextract.extract(url).tld,tldextract.extract(url).domain
        else:
            lang_code='en'
            tld = tldextract.extract(row).tld
            domain = tldextract.extract(row).domain
        language.append(lang_code)
        tlds.append(tld)
        domains.append(domain)
        langtlds.append((lang_code,tld))
        langdomains.append((lang_code,domain))

Output all the dictionaries and txt files

In [26]:
lang_list = make_counter(language)
write_counter(lang_list,'languages')

url_count = make_counter(tlds)
write_counter(url_count,'TLDs')

domain_count = make_counter(domains)
write_counter(domain_count,'domains')

langtlds_count = make_counter(langtlds)
write_counter(langtlds_count,'langtlds')

langdomains_count = make_counter(langdomains)
write_counter(langdomains_count,'langdomains')